In [2]:
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import os

import pandas as pd
pd.options.mode.chained_assignment = None # avoids assignment warning
import numpy as np
import random
from glob import glob
from tqdm import tqdm
tqdm.pandas()  # enable progress bars in pandas operations
import gc

import librosa
import sklearn
import json

# Import for visualization
import matplotlib as mpl
#cmap = mpl.cm.get_cmap('coolwarm')
import matplotlib.pyplot as plt
import librosa.display as lid
import IPython.display as ipd
# import cv2

# Import KaggleDatasets for accessing Kaggle datasets
# from kaggle_datasets import KaggleDatasets

# WandB for experiment tracking
# import wandb

import torchaudio
# import plotly.express as px
from IPython.display import Audio
# from shapely.geometry import Point

# import plotly.express as px

from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import torch


In [3]:
from sklearn.preprocessing import LabelEncoder

In [7]:
!ls ../../data/2024

eBird_Taxonomy_v2021.csv  test_soundscapes  train_metadata.csv
sample_submission.csv	  train_audio	    unlabeled_soundscapes


In [9]:
df = pd.read_csv("../../data/2024/train_metadata.csv")
df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg


In [10]:
# df['secondary_labels'].unique()

In [11]:
lb = LabelEncoder()
lb.fit(df['primary_label'])

LabelEncoder()

In [12]:
lb.classes_[:10], lb.transform(['asbfly'])

(array(['asbfly', 'ashdro1', 'ashpri1', 'ashwoo2', 'asikoe2', 'asiope1',
        'aspfly1', 'aspswi1', 'barfly1', 'barswa'], dtype=object),
 array([0]))

In [13]:
# just checking if all secondary labels are in lb
c = 0
for sc in df.primary_label.unique():
    try:
        lb.transform([sc])
    except:
        c += 1
#         print(f"There is no {sc} in Label Encoder")
N = df.primary_label.nunique()
print(f'There is no {c} classes from {N} secondary labels!')

There is no 0 classes from 182 secondary labels!


In [14]:
# just checking if all secondary labels are in lb
c = 0
for sc in df.secondary_labels.unique():
    try:
        lb.transform([sc])
    except:
        c += 1
#         print(f"There is no {sc} in Label Encoder")
N = df.secondary_labels.nunique()
print(f'There is no {c} classes from {N} secondary labels!')

There is no 491 classes from 491 secondary labels!


In [15]:
df['labels'] = lb.transform(df['primary_label'])
df['labels'].head(5), df['labels'].nunique()

(0    0
 1    0
 2    0
 3    0
 4    0
 Name: labels, dtype: int64,
 182)

In [33]:
train_path = '../../data/2024/train_audio/'
data, rate = torchaudio.load(train_path + df.filename[13])
display(Audio(data[0, :rate*5], rate=rate))

In [34]:
data.shape[1]/rate

38.58284375

In [56]:
def _pad_tensor(x, length):
    _pad = 0.0
    assert x.ndim == 2
    x = np.pad(x, [[0, 0], [0, length - x.shape[1]]], mode="constant", constant_values=_pad)
    return x


def prepare_tensor(inputs, out_steps=1):
    max_len = max((x.shape[1] for x in inputs))
    remainder = max_len % out_steps
    pad_len = max_len + (out_steps - remainder) if remainder > 0 else max_len
    return np.stack([_pad_tensor(x, pad_len) for x in inputs])


# S = librosa.feature.melspectrogram(
#             y=signal,
#             sr=sr,
#             n_fft=N_FFT,
#             hop_length=HOP_SIZE,
#             n_mels=N_MELS,
#             htk=True,
#             fmin=fmin,
#             fmax=fmax,
#         )
#         D = librosa.power_to_db(S ** 2, ref=np.max)


class SimpleDataset(Dataset):
    def __init__(self, df, segment_size = 5, lazy_load = False):
        self.df = df
        self.segment_size = segment_size # Size of random slices in seconds
        self.lazy_load = lazy_load
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        class_id = self.df.labels.values[idx]
        
        if(self.lazy_load):
            melspec = np.load(self.df.filename.values[idx])
        else:
            y, sr = librosa.load('../../data/2024/train_audio/' + df.filename.values[idx], sr=None)
            y = np.asarray(y, dtype=np.float32)

            L_max = len(y)
            if(L_max > self.segment_size*sr):
                R_start = np.random.randint(0,L_max - self.segment_size*sr)
                y_ = y[R_start:R_start+self.segment_size*sr]
            else:
                y_ = y


            melspec = librosa.feature.melspectrogram(y=y_,sr=sr,n_fft=1024,win_length=1024,
                                                     hop_length=256,n_mels=80, fmin = 300,
                                                    fmax = 16000).astype("float32")
            melspec = librosa.power_to_db(melspec ** 2, ref=np.max)
        
        return melspec, class_id

def my_collate_fn(batch):

    melspec = [x[0] for x in batch]
    label = [x[1] for x in batch]
    label = np.array(label)

    melspec = prepare_tensor(melspec)
    
#     print(type(label))

    return torch.FloatTensor(melspec), torch.LongTensor(label)

In [50]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size = 0.01, random_state = 42, stratify=df['labels'])
df_train.shape, df_val.shape

((24214, 13), (245, 13))

In [23]:
from torch.utils.data import DataLoader

train_dataset = SimpleDataset(df_train)
val_dataset = SimpleDataset(df_val)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=my_collate_fn, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=my_collate_fn, num_workers=4)

batch = next(iter(train_dataloader))
batch[0].shape, batch[1].shape

(torch.Size([64, 80, 626]), torch.Size([64]))

In [36]:
%%time
batch = next(iter(train_dataloader))
batch[0].shape, batch[1].shape

CPU times: user 25.8 ms, sys: 133 ms, total: 159 ms
Wall time: 25.8 s


(torch.Size([64, 80, 626]), torch.Size([64]))

In [29]:
mel, styleid = train_dataset.__getitem__(6)
# IPython.display.Audio(y_perturbed, rate = 22050)
mel.shape

(80, 626)

In [37]:
mel.max(), mel.min()

(0.0, -80.0)

# Reading from pre processed npy files



In [61]:
import os

prd_path = '../../preprocessed_data/simple_5sec_spectrogram/'

len(os.listdir(prd_path))

182

In [71]:
labels = []
filenames = []

for folder in os.listdir(prd_path):
    for file in os.listdir(prd_path + folder):
        labels.append(lb.transform([folder])[0])
        filenames.append(prd_path + folder + '/' + file)
        
        if(not os.path.isfile(filenames[-1])):
            break

In [72]:
prd_df = pd.DataFrame({'filename':filenames, 'labels': labels})
prd_df.shape

(146651, 2)

In [97]:
from torch.utils.data import DataLoader


from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(prd_df, test_size = 0.01, random_state = 42, stratify=prd_df['labels'])

from torch.utils.data import DataLoader

train_dataset = SimpleDataset(df_train, lazy_load = True)
val_dataset = SimpleDataset(df_val, lazy_load = True)

train_dataloader = DataLoader(train_dataset, batch_size=182*2, shuffle=True, collate_fn=my_collate_fn, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=my_collate_fn, num_workers=4)

batch = next(iter(train_dataloader))
batch[0].shape, batch[1].shape

(torch.Size([364, 128, 288]), torch.Size([364]))

In [98]:
%%time
batch = next(iter(train_dataloader))
batch[0].shape, batch[1].shape

CPU times: user 522 ms, sys: 205 ms, total: 728 ms
Wall time: 728 ms


(torch.Size([364, 128, 251]), torch.Size([364]))

In [99]:
from torch import nn
import torch
import torch.nn.functional as F
import torch.optim as optim

class ReferenceEncoderClassifier(nn.Module):
    """NN module creating a fixed size prosody embedding from a spectrogram.

    inputs: mel spectrograms [batch_size, num_spec_frames, num_mel]
    outputs: [batch_size, embedding_dim]
    """

    def __init__(self, num_mel, embedding_dim, num_classes, use_nonlinear_proj = False):

        super().__init__()
        self.num_mel = num_mel
        filters = [1] + [32, 32, 64, 64, 128, 128]
        num_layers = len(filters) - 1
        convs = [
            nn.Conv2d(
                in_channels=filters[i], out_channels=filters[i + 1], kernel_size=(3, 3), stride=(2, 2), padding=(1, 1)
            )
            for i in range(num_layers)
        ]
        self.convs = nn.ModuleList(convs)
        self.bns = nn.ModuleList([nn.BatchNorm2d(num_features=filter_size) for filter_size in filters[1:]])

        post_conv_height = self.calculate_post_conv_height(num_mel, 3, 2, 1, num_layers)
        self.recurrence = nn.GRU(
            input_size=filters[-1] * post_conv_height, hidden_size=embedding_dim, batch_first=True
        )

        self.dropout = nn.Dropout(p=0.5)

        self.use_nonlinear_proj = use_nonlinear_proj

        if(self.use_nonlinear_proj):
            self.proj = nn.Linear(embedding_dim, embedding_dim)
            nn.init.xavier_normal_(self.proj.weight) # Good init for projection
            # self.proj.bias.data.zero_() # Not random bias to "move" z

        self.classifier_layer = nn.Linear(embedding_dim, num_classes)

    def forward(self, inputs):
        batch_size = inputs.size(0)
        x = inputs.view(batch_size, 1, -1, self.num_mel)
        # x: 4D tensor [batch_size, num_channels==1, num_frames, num_mel]
        for conv, bn in zip(self.convs, self.bns):
            x = conv(x)
            x = bn(x)
            x = F.relu(x)

        x = x.transpose(1, 2)
        # x: 4D tensor [batch_size, post_conv_width,
        #               num_channels==128, post_conv_height]
        post_conv_width = x.size(1)
        x = x.contiguous().view(batch_size, post_conv_width, -1)
        # x: 3D tensor [batch_size, post_conv_width,
        #               num_channels*post_conv_height]
        self.recurrence.flatten_parameters()
        _, out = self.recurrence(x)
        # out: 3D tensor [seq_len==1, batch_size, encoding_size=128]

        if(self.use_nonlinear_proj):
            out = torch.tanh(self.proj(out))
            out = self.dropout(out)

        out = self.classifier_layer(out)

        return out.squeeze(0)

    @staticmethod
    def calculate_post_conv_height(height, kernel_size, stride, pad, n_convs):
        """Height of spec after n convolutions with fixed kernel/stride/pad."""
        for _ in range(n_convs):
            height = (height - kernel_size + 2 * pad) // stride + 1
        return height

In [100]:
from tqdm import tqdm
def train(model, train_loader, val_loader, epochs = 100, eval_epochs = 5, lr = 0.001, use_cuda = True, save_model_path = 'baseline'):
    if(use_cuda):
        model_ = model.cuda()
    else:
        return 'If you are really wanting to train it on CPU code it itself!'

    optimizer = optim.Adam(model_.parameters(), lr=lr)

    # Defining losses
    ce = nn.CrossEntropyLoss()
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []

    best_loss = 9999

    niter = epochs

    print(f"Start training with {niter} epochs: \n")
    iter_ = 0
    for _ in range(0, niter):
        model_.train()
        print(f'EPOCH {_}/{niter}')
        acc_ = 0
        acc_b = 0
        acc_gd = 0
        acc_gb = 0
        batch_len = 0
        count_upt = 0

        for batch in tqdm(train_loader):

            melspec = batch[0].cuda()
            labels = batch[1].cuda()
            optimizer.zero_grad()
            predictions = model_(melspec)

            loss = ce(predictions, labels)
            train_loss.append(loss.item())
            acc_ += (predictions.argmax(axis = -1).cpu() == labels.cpu()).sum()

            # Total loss and opt step
            loss.backward()
            optimizer.step()

            batch_len += len(labels)
            iter_+=1

            if iter_ % 5000 == 0:
            #       if(loss < best_loss):
            #         print("Saving best model")
                torch.save(model_.state_dict(), f'{save_model_path}/checkpoint_{iter_}.pth')
            #         best_loss = loss
            #         count_upt += 1



        train_acc.append(acc_/batch_len)

        if(_%eval_epochs == 0):
            model_.eval()
            acc_ = 0
            batch_len_ = 0
            eval_loss = 0
            for batch in tqdm(val_loader):

                melspec = batch[0].cuda()
                labels = batch[1].cuda()
                optimizer.zero_grad()
                predictions = model_(melspec)
                batch_len_ += len(labels)
                loss = ce(predictions, labels)
                val_loss.append(loss.item())
                acc_ += (predictions.argmax(axis = -1).cpu() == labels.cpu()).sum()
                eval_loss += loss.item()

            eval_avg_loss = eval_loss / len(val_loader)

            print(f'EVAL EPOCH ACC = {acc_/batch_len_}')
            
            if(eval_avg_loss < best_loss):
                print("Saving best model")
                torch.save(model_.state_dict(), f'{save_model_path}/best_model_{iter_}.pth')
                best_loss = eval_avg_loss
                count_upt += 1

            val_acc.append(acc_/batch_len_)


    return model_, train_loss, val_loss, train_acc, val_acc


In [ ]:
model = ReferenceEncoderClassifier(128,128,182)
model, train_loss, val_loss, train_acc, val_acc = train(model, train_dataloader, val_dataloader, epochs = 15, eval_epochs = 1, lr = 0.001, use_cuda = True, save_model_path = './chpks/simple_v0/')

Start training with 15 epochs: 

EPOCH 0/15


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 34.69it/s]


EVAL EPOCH ACC = 0.13360600173473358
Saving best model
EPOCH 1/15


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 36.66it/s]


EVAL EPOCH ACC = 0.12678936123847961
EPOCH 2/15


 25%|█████████████████████████▊                                                                              | 99/399 [00:11<00:33,  8.95it/s]

In [ ]:
print(train_acc)

In [ ]:
print(val_acc)